<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/CSC645/blob/master/shallow_tensorflow_answer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using Tensorflow to model the shallow network
In this exercise will we redo, using tensorflow the shallow network that we trained from first
principles before to recognize the "flower" shape.


In [1]:
import tensorflow as tf
import numpy as np
from google.colab import files
file=files.upload()

Saving sonar.csv to sonar (14).csv


In [2]:
import pandas as pd
df=pd.read_csv("sonar.csv")


In [3]:
m=df.values
np.random.shuffle(m)
X=m[:,0:60].astype("float32")
Y=m[:,60]
Y=np.array([1.0 if i=='M' else 0.0 for i in Y])
Y=Y.reshape((len(Y),1))
Y=Y.astype("float32")
print(type(Y[0,0]))

x_train=X[0:150]
x_test=X[150:208]
y_train=Y[0:150]
y_test=Y[150:208]

<class 'numpy.float32'>


### Reading the data
First recall that tensorflow stacks the samples row-wise instead of column-wise
as we have been doing when we did the gradient descent oursleves. Therefore in the last line of the
function load_dataset() below we don't take the transpose of X and Y as we did before.

### Defining the parameters
Below we define the parameters that are needed. We know that n_x=2 and n_y=1 but we extract them from the shape of
X_data and Y_data after we call load_dataset() . We also set the number
of data points to 500.

In [4]:
learning_rate = 3
nb_iterations = 1500

# Network Parameters
#n_h = 4 # number of neurons in hidden layer
n_h = 64 # number of neurons in hidden layer

n_x = X.shape[1] #number of neurons in input
n_y = Y.shape[1] #number of neurons in ouput


### Initialization

Since tensorflow stacks the data row-wise the forward propagation is slightly different then we are used to.
Let $W^1$,$W^2$,$b^1$,$b^2$ be the weights and biases of the first and second layer respectively then forward propagation is define as
\begin{align*}
Z^1&=X\cdot W^1+b^1\\
A^1 &=\sigma(Z^1)\\
Z^2 &=A^1\cdot W^2+b2\\
A^2 &=\sigma(Z^2)
\end{align*}
Accorging to the above equations you have to define the tensorflow variables that will hold the weights and biases. 
The biases are set to zero using the tensorflow function tf.zeros([size]) and the weights randomly using tf.random_normal([size1,size2]) using the appropriate sizes.
Also we have to define two placeholders for the data X and Y

In [5]:

initializer = tf.initializers.RandomNormal()

W1=tf.Variable(initializer([n_x,n_h]),trainable=True,dtype=tf.float32)
W2=tf.Variable(initializer([n_h,n_y]),trainable=True,dtype=tf.float32)

b1=tf.Variable(tf.zeros([n_h]))            #biases of the first layer
b2=tf.Variable(tf.zeros([n_y]))            #biases of the second layer
print(n_y)

1


### Defining the model
Our model has two layers. The function "model" below should return the ouput of our model for a given input.

In [6]:
def model(input):
    # Hidden fully connected layer with 256 neurons
   
    layer_1 = tf.add(tf.matmul(input, W1), b1)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(tf.sigmoid(layer_1), W2) + b2
    return out_layer

Once the model is defined the remaining code is similar to our previous exercise. We define the loss
as an average over the cross-entropy but this time since it is binary classification we use the sigmoid instead
of the softmax function. Then our optimizer uses gradient descent to minimize the loss

In [7]:

# Define loss and optimize
def loss(pred,label):
   return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=label))


The model is defined now we run our computation in a session.

In [8]:
# Initializing the variables
optimizer=tf.optimizers.SGD(learning_rate)
def prediction(X):
  a=tf.math.sigmoid(model(X))
  return tf.cast((a>0.5),tf.int32)
def train(data,labels):
  with tf.GradientTape() as tape:
    diff=loss(model(data),labels)
  grad=tape.gradient(diff,[W1,W2,b1,b2])
  optimizer.apply_gradients( zip( grad , [W1,W2,b1,b2] ) )
  pT=tf.transpose(prediction(data))
  correct=np.squeeze(np.dot(pT,labels)+np.dot(1-pT,1-labels))
  return diff,correct

for i in range(nb_iterations):
 cost,corr=train(x_train,y_train)
 if(i%500==0):
  print("cost={},accuracy={}/{}".format(cost,corr,x_train.shape[0]))
  
 


pT=tf.transpose(prediction(x_test))
print(np.dot(pT,y_test))
print(np.dot(1-pT,1-y_test))
correct=np.dot(pT,y_test)+np.dot(1-pT,1-y_test)
accuracy=100*float(np.squeeze(correct))/float(y_test.shape[0])
print("Accuracy="+str(accuracy))


cost=0.6937292218208313,accuracy=71.0/150
cost=0.31074628233909607,accuracy=133.0/150
cost=0.0672723799943924,accuracy=148.0/150
[[28.]]
[[18.]]
Accuracy=80.70175438596492
